In [1]:
!pip install fuzzywuzzy[speedup]


DEPRECATION: Loading egg at /srv/conda/envs/notebook/lib/python3.11/site-packages/odp-0.0.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
  Using cached fuzzywuzzy-0.18.0-py2.py3-none-any.whl.metadata (4.9 kB)
  Using cached python_Levenshtein-0.25.1-py3-none-any.whl.metadata (3.7 kB)
  Using cached Levenshtein-0.25.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.3 kB)
  Using cached rapidfuzz-3.9.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
Using cached python_Levenshtein-0.25.1-py3-none-any.whl (9.4 kB)
Using cached Levenshtein-0.25.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (177 kB)
Using cached fuzzywuzzy-0.18.0-py2.py3-none-any.whl (18 kB)
Using cached rapidfuzz-3.9.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.4 MB)


In [2]:
import pandas as pd
import ast
import json
from fuzzywuzzy import process

### Load classification from the Accenture modelling work

In [76]:
# Load the JSON file
with open('Dataset_and_Org_classifications/AccentureOBISclassification_dict_complete_3_updated.json') as f:
    data = json.load(f)

In [77]:
# Convert JSON data to pandas DataFrame
df_AccentureClass = pd.DataFrame.from_dict(data, orient='index').reset_index(names='org')


In [78]:
df_AccentureClass

,org,classification,source
0,US National Oceanographic Data Center,public,oceanexpert_id
1,Thünen Institute of Sea Fisheries,public,oceanexpert_id
2,Organización para la Sustentabilidad y Conserv...,public,rag
3,University of the West Indies,public,baseline
4,Università degli Studi di Pavia Centro Interdi...,public,oceanexpert_id
...,...,...,...
2569,Utrecht University,public,oceanexpert_id
2570,"Laboratoire d'Ecologie du Benthos, Faculté des...",public,rag
2571,"Sorbonne Université; Sorbonne Université/CNRS,...",public,oceanexpert_id
2572,Aalborg University; Department of Culture and ...,public,oceanexpert_id


In [10]:
sorted(set(df_AccentureClass['org'].dropna().apply(lambda x: x.strip())), key=len)[:10]


['NA', 'ND', 'GHD', 'MIT', 'DFO', 'ANU', 'UCT', 'N/A', 'IRD', 'ANH']

In [9]:
df_AccentureClass = df_AccentureClass[~df_AccentureClass['org'].isin(['c', '/'])]


In [12]:
df_OBISorgInfo = pd.read_csv('Dataset_and_Org_classifications/OBIS_DatasetID_2_orgnames.csv')

In [13]:
df_OBISorgInfo['Data providers']

0                            ['University of Queensland']
1       ['CSIRO National Collections and Marine Infras...
2       ['The Australian National University', 'CSIRO ...
3       ['Delaware Museum of Nature and Science – Moll...
4       ['Invert-E-Base Portal', 'Malacology Collectio...
                              ...                        
5318    ['Australian Antarctic Division', 'Intergovern...
5319                                                   []
5320                                                   []
5321                                                   []
5322                                                   []
Name: Data providers, Length: 5323, dtype: object

In [14]:
df_OBISorgInfo[df_OBISorgInfo['Data providers'] == "[]"]

,dataset_id,Data providers,Contact Organizations,Total Occurrences
111,6dc643dc-1a98-4292-b1f6-483ed5659146,[],"['Happywhale', 'Happywhale', 'Marine Geospatia...",0
112,fbe8a17d-9734-471c-98f5-98c837c49bf4,[],"['Happywhale', 'Happywhale', 'Marine Geospatia...",0
113,f49b2ebe-d278-4454-8b3e-c04eb3c0362d,[],"['Happywhale', 'Happywhale', 'Marine Geospatia...",0
115,9080ddf4-075c-4bf1-875c-338998c7f897,[],"['Happywhale', 'Happywhale', 'Marine Geospatia...",0
116,6cccff93-336f-4f42-8af3-338e6ac3e534,[],"['Happywhale', 'Happywhale', 'Marine Geospatia...",0
...,...,...,...,...
5316,691cee6f-19ee-4261-86e0-8e5ffcdfd4c9,[],[],0
5319,cf494b72-7b4d-4ca6-bb90-6a96554dcd57,[],[],0
5320,424b6232-cb9a-489a-8894-30fd7b951d49,[],[],0
5321,ddf1a66a-6913-4653-9f3a-44072705ad64,[],[],0


In [15]:
df_OBISorgInfo_cleaned = df_OBISorgInfo[df_OBISorgInfo['Total Occurrences'] != 0].reset_index(drop=True)
df_OBISorgInfo_cleaned

,dataset_id,Data providers,Contact Organizations,Total Occurrences
0,d6ae6af8-8eac-4fbd-9ce5-b49b1137351c,['University of Queensland'],"['University of Queensland', 'University of Qu...",3665
1,96041282-9386-4a8a-a564-fd753f1222b0,['CSIRO National Collections and Marine Infras...,"['CSIRO Environment', 'CSIRO Environment', 'CS...",19763
2,61dc18bb-e2ec-43a1-949c-120d58235b61,"['The Australian National University', 'CSIRO ...","['Australian National University', 'Australian...",1960
3,0c7b3f51-33c8-494c-ab8a-c5bd28f40f87,['Delaware Museum of Nature and Science – Moll...,"['Invert-E-Base Portal', 'Delaware Museum of N...",34280
4,8ea620e9-37d5-4ff6-9a02-0bc94356c2d5,"['Invert-E-Base Portal', 'Malacology Collectio...","['Invert-E-Base Portal', 'Malacology Collectio...",164632
...,...,...,...,...
5008,7aee17f1-d6e8-4204-a4b2-eafdf89563cf,['Intergovernmental Oceanographic Commission o...,['National Oceanic and Atmospheric Administrat...,229575
5009,ce92b079-6389-4880-99d8-1a97fb171211,['Intergovernmental Oceanographic Commission o...,['Intergovernmental Oceanographic Commission o...,835
5010,4a4bc032-e738-4102-b67f-445578b656d9,['Intergovernmental Oceanographic Commission o...,['South Carolina Department of Natural Resourc...,1709
5011,63108715-3d0c-4a26-847f-16ca305c9cb1,['Intergovernmental Oceanographic Commission o...,['U.S. Geological Survey Southeast Ecological ...,908


In [48]:
df_OBISorgInfo_filtered = df_OBISorgInfo_cleaned[df_OBISorgInfo_cleaned['Data providers'] != '[]'].reset_index(drop=True)
df_OBISorgInfo_filtered = df_OBISorgInfo_filtered.drop(columns=['Contact Organizations'])
df_OBISorgInfo_filtered

,dataset_id,Data providers,Total Occurrences
0,d6ae6af8-8eac-4fbd-9ce5-b49b1137351c,['University of Queensland'],3665
1,96041282-9386-4a8a-a564-fd753f1222b0,['CSIRO National Collections and Marine Infras...,19763
2,61dc18bb-e2ec-43a1-949c-120d58235b61,"['The Australian National University', 'CSIRO ...",1960
3,0c7b3f51-33c8-494c-ab8a-c5bd28f40f87,['Delaware Museum of Nature and Science – Moll...,34280
4,8ea620e9-37d5-4ff6-9a02-0bc94356c2d5,"['Invert-E-Base Portal', 'Malacology Collectio...",164632
...,...,...,...
5003,7aee17f1-d6e8-4204-a4b2-eafdf89563cf,['Intergovernmental Oceanographic Commission o...,229575
5004,ce92b079-6389-4880-99d8-1a97fb171211,['Intergovernmental Oceanographic Commission o...,835
5005,4a4bc032-e738-4102-b67f-445578b656d9,['Intergovernmental Oceanographic Commission o...,1709
5006,63108715-3d0c-4a26-847f-16ca305c9cb1,['Intergovernmental Oceanographic Commission o...,908


In [49]:
df_OBISorgInfo = df_OBISorgInfo_filtered
df_OBISorgInfo

,dataset_id,Data providers,Total Occurrences
0,d6ae6af8-8eac-4fbd-9ce5-b49b1137351c,['University of Queensland'],3665
1,96041282-9386-4a8a-a564-fd753f1222b0,['CSIRO National Collections and Marine Infras...,19763
2,61dc18bb-e2ec-43a1-949c-120d58235b61,"['The Australian National University', 'CSIRO ...",1960
3,0c7b3f51-33c8-494c-ab8a-c5bd28f40f87,['Delaware Museum of Nature and Science – Moll...,34280
4,8ea620e9-37d5-4ff6-9a02-0bc94356c2d5,"['Invert-E-Base Portal', 'Malacology Collectio...",164632
...,...,...,...
5003,7aee17f1-d6e8-4204-a4b2-eafdf89563cf,['Intergovernmental Oceanographic Commission o...,229575
5004,ce92b079-6389-4880-99d8-1a97fb171211,['Intergovernmental Oceanographic Commission o...,835
5005,4a4bc032-e738-4102-b67f-445578b656d9,['Intergovernmental Oceanographic Commission o...,1709
5006,63108715-3d0c-4a26-847f-16ca305c9cb1,['Intergovernmental Oceanographic Commission o...,908


In [50]:
# Convert strings to actual lists
df_OBISorgInfo['Data providers'] = df_OBISorgInfo['Data providers'].apply(ast.literal_eval)

In [56]:
# Extract unique data providers
unique_data_providers = set(org.strip() for sublist in df_OBISorgInfo['Data providers'] for org in sublist if org.strip() not in ['c', '/', ''])

In [57]:
len(unique_data_providers)

2467

In [58]:
sorted(unique_data_providers, key=len)[:10]


['NA', 'ND', 'GHD', 'MIT', 'DFO', 'ANU', 'UCT', 'N/A', 'IRD', 'ANH']

In [59]:
df_OBISorgInfo['Data providers'].apply(lambda x: sorted(set(org.strip() for org in x if org.strip()), key=len)[:10])

0                              [University of Queensland]
1       [CSIRO Environment, CSIRO National Collections...
2       [The Australian National University, CSIRO Nat...
3       [Invert-E-Base Portal, Delaware Museum of Natu...
4       [Invert-E-Base Portal, Malacology Collection a...
                              ...                        
5003    [Intergovernmental Oceanographic Commission of...
5004    [Centro de Referência em Informação Ambiental ...
5005    [Intergovernmental Oceanographic Commission of...
5006    [U.S. Geological Survey HQ, Intergovernmental ...
5007    [Australian Antarctic Division, Intergovernmen...
Name: Data providers, Length: 5008, dtype: object

In [60]:
# Step 2: Create a set of entries from df_AccentureClass['org']
orgs_in_df_AccentureClass = set(df_AccentureClass['org'].str.strip())

# Step 3: Remove any entries from unique_data_providers that match entries in df_AccentureClass['org']
filtered_unique_data_providers = unique_data_providers - orgs_in_df_AccentureClass


In [61]:
sorted(orgs_in_df_AccentureClass, key=len)[:10]


['NA', 'ND', 'GHD', 'MIT', 'DFO', 'ANU', 'UCT', 'N/A', 'IRD', 'ANH']

In [62]:
print(len(filtered_unique_data_providers))

45


print(len(filtered_unique_data_providers)) should be 45

In [63]:
sorted(filtered_unique_data_providers, key=len)[:20]


['MHNMC',
 'OBXCDR',
 'FL-FAU-HBOI',
 'Corpoguajira',
 'Burke Museum',
 'Centre ORSTOM',
 'Dolphin Encounter',
 'Cornell University',
 'Invert-E-Base Portal',
 'Universiteit Amsterdam',
 "Centre de l'Environnement",
 'New Zealand Sea Adventures',
 'University of Central Florida',
 "Centre d'Oceanologie de Marseille",
 'Integrated Marine Observing System',
 'Boston University, Department of Biology',
 'Canadian Wildlife Service - Quebec Region',
 'Department of Zoology, University of Otago',
 'Leibniz Center for Tropical Marine Ecology',
 'Bureau Waardenburg bv – Ecology & Landscape']

In [64]:
#####################################################

# 2. Create a dictionary to hold the fuzzy mappings
org_mapping = {}

# Define a similarity threshold (e.g., 80%)
similarity_threshold = 90

# 3. Perform fuzzy matching for the remaining organizations
for org in filtered_unique_data_providers:
        # Get the best matches from the remaining all_orgs
        best_matches = process.extract(org, df_AccentureClass['org'], limit=1)
        
        # Filter matches that meet the threshold
        filtered_matches = [match for match, score, index in best_matches if score >= similarity_threshold]
        
        # Add the mapping to the dictionary
        org_mapping[org] = filtered_matches

# 4. Display the final mapping
for source_org, matched_orgs in org_mapping.items():
    print(f"Source: {source_org}")
    print(f"Matches: {matched_orgs}")
    print("-----")

Source: Department of Zoology, University of Otago
Matches: ['University of Otago']
-----
Source: Corpoguajira
Matches: ['Corporación Autónoma Regional de La Guajira - Corpoguajira']
-----
Source: School of Environmental and Marine Sciences, University of Auckland
Matches: ['University of Auckland']
-----
Source: School of Biological Sciences, University of Canterbury
Matches: ['University of Canterbury']
-----
Source: OBXCDR
Matches: []
-----
Source: University of California Santa Cruz, Department of Ecology and Evolutionary Biology
Matches: ['Department of Ecology and Evolutionary Biology, University of California Santa Cruz']
-----
Source: FL-FAU-HBOI
Matches: []
-----
Source: Laboratoire de Biologie Marine et Malacologie, Ecole Pratique des Hautes Etudes
Matches: []
-----
Source: Dolphin Encounter
Matches: []
-----
Source: Leibniz Center for Tropical Marine Ecology
Matches: []
-----
Source: Marine Conservation Unit, Department of Conservation
Matches: ['Department of Conservation']

In [65]:
# 3. Update df_OBISorgInfo_filtered with correct terminology from the MATCH list
def update_data_providers(row):
    updated_providers = []
    for provider in row['Data providers']:
        # If the provider matches any key in org_mapping, use the mapped value
        for source_org, matched_orgs in org_mapping.items():
            if provider in matched_orgs:
                updated_providers.append(source_org)
                break
        else:
            updated_providers.append(provider)
    return updated_providers

df_OBISorgInfo_filtered['Data providers'] = df_OBISorgInfo_filtered.apply(update_data_providers, axis=1)

In [66]:
df_OBISorgInfo_filtered

,dataset_id,Data providers,Total Occurrences
0,d6ae6af8-8eac-4fbd-9ce5-b49b1137351c,[University of Queensland],3665
1,96041282-9386-4a8a-a564-fd753f1222b0,[CSIRO National Collections and Marine Infrast...,19763
2,61dc18bb-e2ec-43a1-949c-120d58235b61,"[The Australian National University, CSIRO Nat...",1960
3,0c7b3f51-33c8-494c-ab8a-c5bd28f40f87,[Delaware Museum of Nature and Science – Mollu...,34280
4,8ea620e9-37d5-4ff6-9a02-0bc94356c2d5,"[Invert-E-Base Portal, Malacology Collection a...",164632
...,...,...,...
5003,7aee17f1-d6e8-4204-a4b2-eafdf89563cf,[Intergovernmental Oceanographic Commission of...,229575
5004,ce92b079-6389-4880-99d8-1a97fb171211,[Intergovernmental Oceanographic Commission of...,835
5005,4a4bc032-e738-4102-b67f-445578b656d9,[Intergovernmental Oceanographic Commission of...,1709
5006,63108715-3d0c-4a26-847f-16ca305c9cb1,[Intergovernmental Oceanographic Commission of...,908


In [67]:
# 4. Add new entries to df_AccentureClass for sources with no match
matched_orgs = set(org for matches in org_mapping.values() for org in matches)
unmatched_sources = [org for org in filtered_unique_data_providers if org not in matched_orgs]

# Create DataFrame for new entries
new_entries = pd.DataFrame({
    'org': unmatched_sources,
    'classification': 'unknown(public)',
    'source': 'manual_MW'
})

# Append new entries to df_AccentureClass
df_AccentureClass_updated = pd.concat([df_AccentureClass, new_entries], ignore_index=True)

In [68]:
df_AccentureClass_updated

,org,classification,source
0,US National Oceanographic Data Center,public,oceanexpert_id
1,Thünen Institute of Sea Fisheries,public,oceanexpert_id
2,Organización para la Sustentabilidad y Conserv...,public,rag
3,University of the West Indies,public,baseline
4,Università degli Studi di Pavia Centro Interdi...,public,oceanexpert_id
...,...,...,...
2612,Burke Museum,unknown(public),manual_MW
2613,US National Oceanographic Data Center (NODC),unknown(public),manual_MW
2614,"University of California, San Diego, Scripps I...",unknown(public),manual_MW
2615,U.S. Geological Survey – Patuxent Wildlife Res...,unknown(public),manual_MW


In [69]:
# Create a classification lookup dictionary from df_AccentureClass_updated
classification_lookup = df_AccentureClass_updated.set_index('org')['classification'].to_dict()

# Function to calculate the metrics
def calculate_metrics(row):
    providers = row['Data providers']
    
    # Get the classifications for the providers
    classifications = [classification_lookup.get(provider, 'unknown') for provider in providers]
    
    # Calculate flags and percentages
    private_flag = any(classification == 'private' for classification in classifications)
    total_providers = len(providers)
    
    if total_providers == 0:
        return {
            'PrivateFlag': private_flag,
            'PercProvPRIVATE': 0.0,
            'PercProvPUBLIC': 0.0,
            'PercProvUNKNOWNPUBLIC': 0.0,
            'PercProvOUTLIER': 0.0,
            'WeightedPercOccurrencesPrivate': 0.0
        }
    
    perc_prov_private = classifications.count('private') / total_providers * 100
    perc_prov_public = classifications.count('public') / total_providers * 100
    perc_prov_unknown_public = classifications.count('unknown(public)') / total_providers * 100
    perc_prov_outlier = classifications.count('outlier') / total_providers * 100
    
    # Calculate weighted percentage occurrences private
    weighted_perc_occurrences_private = row['Total Occurrences'] * (perc_prov_private / 100)
    
    return {
        'PrivateFlag': private_flag,
        'PercProvPRIVATE': perc_prov_private,
        'PercProvPUBLIC': perc_prov_public,
        'PercProvUNKNOWNPUBLIC': perc_prov_unknown_public,
        'PercProvOUTLIER': perc_prov_outlier,
        'WeightedPercOccurrencesPrivate': weighted_perc_occurrences_private
    }

# Apply the function to each row
metrics = df_OBISorgInfo_filtered.apply(calculate_metrics, axis=1, result_type='expand')

# Join metrics with df_OBISorgInfo_filtered
df_OBISorgInfo_filtered_metrics = df_OBISorgInfo_filtered.join(metrics)

In [70]:
df_OBISorgInfo_filtered_metrics

,dataset_id,Data providers,Total Occurrences,PrivateFlag,PercProvPRIVATE,PercProvPUBLIC,PercProvUNKNOWNPUBLIC,PercProvOUTLIER,WeightedPercOccurrencesPrivate
0,d6ae6af8-8eac-4fbd-9ce5-b49b1137351c,[University of Queensland],3665,False,0.0,100.000000,0.000000,0.0,0.0
1,96041282-9386-4a8a-a564-fd753f1222b0,[CSIRO National Collections and Marine Infrast...,19763,False,0.0,100.000000,0.000000,0.0,0.0
2,61dc18bb-e2ec-43a1-949c-120d58235b61,"[The Australian National University, CSIRO Nat...",1960,False,0.0,100.000000,0.000000,0.0,0.0
3,0c7b3f51-33c8-494c-ab8a-c5bd28f40f87,[Delaware Museum of Nature and Science – Mollu...,34280,False,0.0,0.000000,100.000000,0.0,0.0
4,8ea620e9-37d5-4ff6-9a02-0bc94356c2d5,"[Invert-E-Base Portal, Malacology Collection a...",164632,False,0.0,0.000000,100.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...
5003,7aee17f1-d6e8-4204-a4b2-eafdf89563cf,[Intergovernmental Oceanographic Commission of...,229575,False,0.0,66.666667,33.333333,0.0,0.0
5004,ce92b079-6389-4880-99d8-1a97fb171211,[Intergovernmental Oceanographic Commission of...,835,False,0.0,100.000000,0.000000,0.0,0.0
5005,4a4bc032-e738-4102-b67f-445578b656d9,[Intergovernmental Oceanographic Commission of...,1709,False,0.0,100.000000,0.000000,0.0,0.0
5006,63108715-3d0c-4a26-847f-16ca305c9cb1,[Intergovernmental Oceanographic Commission of...,908,False,0.0,100.000000,0.000000,0.0,0.0


In [71]:
df_OBISorgInfo_filtered_metrics[df_OBISorgInfo_filtered_metrics['PrivateFlag'] == True]

,dataset_id,Data providers,Total Occurrences,PrivateFlag,PercProvPRIVATE,PercProvPUBLIC,PercProvUNKNOWNPUBLIC,PercProvOUTLIER,WeightedPercOccurrencesPrivate
23,a35e13ec-1b51-4763-bf99-9f6d3a6e6bef,"[Duke University, SEATURTLE.ORG, Tracking Sea ...",30,True,33.333333,66.666667,0.000000,0.0,10.000000
27,5f6740b4-817c-402e-a0d9-758e6a8b7ff6,"[Aqua y terra, C&amp;C consultores ambientales...",1576,True,75.000000,25.000000,0.000000,0.0,1182.000000
273,bef38a3a-fc41-4be1-82fa-9aa75ad08b81,"[Vlaams Instituut voor de Zee, Institut França...",10949,True,4.166667,91.666667,4.166667,0.0,456.208333
321,9c27b24d-e80f-4624-8f73-628b8fd7cb43,"[Vlaams Instituut voor de Zee, Deltares, Wagen...",38307,True,25.000000,75.000000,0.000000,0.0,9576.750000
498,5ec42fe0-a902-46e4-b9cb-054c86dc7c8b,"[Vlaams Instituut voor de Zee, Deltares, Haven...",18873,True,20.000000,80.000000,0.000000,0.0,3774.600000
...,...,...,...,...,...,...,...,...,...
4853,1b351842-59b5-402a-bc6d-87bd3d60da0b,"[Duke University, Mare Nostrum NGO, The Agreem...",62,True,33.333333,66.666667,0.000000,0.0,20.666667
4878,7686d78d-fe4f-4e22-81d1-18197b5f76e3,"[International Seabed Authority, China Ocean M...",304,True,50.000000,50.000000,0.000000,0.0,152.000000
4887,49f1cbfd-07ab-422f-8952-e53ff4974d8d,"[Duke University, Mare Nostrum NGO, The Agreem...",99,True,33.333333,66.666667,0.000000,0.0,33.000000
4917,c6e7b1fb-6fb6-43dc-b65a-357aa6ed3d3f,"[Duke University, HDR, Inc.]",117,True,50.000000,50.000000,0.000000,0.0,58.500000


In [72]:
OBIS_dataset_metrics_OUTPUT_FILENAME = 'Dataset_and_Org_classifications/OBIS_dataset_metrics.csv'

OBIS_provider_metrics_OUTPUT_FILENAME = 'Dataset_and_Org_classifications/OBIS_provider_metrics.csv'

In [73]:
df_OBISorgInfo_filtered_metrics.to_csv(OBIS_dataset_metrics_OUTPUT_FILENAME, index=False)

In [74]:
# Create a new DataFrame to store the results
df_OBIS_provider_metrics = df_AccentureClass_updated.copy()

# Initialize new columns
df_OBIS_provider_metrics['Dataset_ids'] = None
df_OBIS_provider_metrics['Occurrences'] = 0
df_OBIS_provider_metrics['WeightedOccurrences'] = 0

# Function to compute dataset_ids, occurrences, and weighted occurrences for each organization
def compute_values(org):
    # Get dataset_ids where the organization is a data provider
    dataset_ids = df_OBISorgInfo_filtered_metrics[df_OBISorgInfo_filtered_metrics['Data providers'].apply(lambda x: org in x)]['dataset_id'].tolist()
    
    # Sum of occurrences
    total_occurrences = df_OBISorgInfo_filtered_metrics[df_OBISorgInfo_filtered_metrics['dataset_id'].isin(dataset_ids)]['Total Occurrences'].sum()
    
    # Sum of weighted occurrences
    weighted_occurrences = df_OBISorgInfo_filtered_metrics[df_OBISorgInfo_filtered_metrics['dataset_id'].isin(dataset_ids)]['WeightedPercOccurrencesPrivate'].sum()
    
    return dataset_ids, total_occurrences, weighted_occurrences

# Apply the function for each organization
df_OBIS_provider_metrics[['Dataset_ids', 'Occurrences', 'WeightedOccurrences']] = df_OBIS_provider_metrics['org'].apply(compute_values).apply(pd.Series)

# Display the resulting DataFrame
print("Resulting DataFrame with Dataset IDs, Occurrences, and Weighted Occurrences:")
df_OBIS_provider_metrics

Resulting DataFrame with Dataset IDs, Occurrences, and Weighted Occurrences:


,org,classification,source,Dataset_ids,Occurrences,WeightedOccurrences
0,US National Oceanographic Data Center,public,oceanexpert_id,[],0,0.0
1,Thünen Institute of Sea Fisheries,public,oceanexpert_id,[],0,0.0
2,Organización para la Sustentabilidad y Conserv...,public,rag,"[481f8003-db1e-435c-b012-91cd84573507, 79708a8...",207,0.0
3,University of the West Indies,public,baseline,[dbb57c74-c34c-4502-b1d0-15324a7fabc4],429,0.0
4,Università degli Studi di Pavia Centro Interdi...,public,oceanexpert_id,"[6c7069ec-4541-4b66-8211-528b60a7dd3d, ca341e2...",220,0.0
...,...,...,...,...,...,...
2612,Burke Museum,unknown(public),manual_MW,[4e88f9c8-868e-4b0a-bc48-3d409a5448fe],148577,0.0
2613,US National Oceanographic Data Center (NODC),unknown(public),manual_MW,[bf78a780-0a48-45d6-af11-cf52b7b6ec5f],564994,0.0
2614,"University of California, San Diego, Scripps I...",unknown(public),manual_MW,[0b7ba344-5ff5-4a72-8e20-43eb86b097da],422000,0.0
2615,U.S. Geological Survey – Patuxent Wildlife Res...,unknown(public),manual_MW,[2a8ddd06-2a18-4fae-81cc-5f5bfe0ef335],374585,0.0


In [75]:
df_OBIS_provider_metrics.to_csv(OBIS_provider_metrics_OUTPUT_FILENAME, index=False)